In [ ]:
# read the file /home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/partial_anisRemesh_shot_lambda_0.01_anisRemesh_holes_partial_0.8_xy/eval/epoch_95/SHREC16_holes_pair_noSingle-test/no_smoothing/log_no_smoothing.txt

with open('/home/s94zalek_hpc/shape_matching/my_code/experiments/ddpm/partial_anisRemesh_shot_lambda_0.01_anisRemesh_holes_partial_0.8_xy/eval/epoch_95/SHREC16_holes_pair_noSingle-test/no_smoothing/log_no_smoothing.txt', 'r') as f:
    lines = f.readlines()

len(lines)

In [ ]:
# find lines like 'Geo error median pairzo: 18.37' and get the number

geo_error = []
for line in lines:
    if 'Geo error median pairzo' in line:
        geo_error.append(float(line.split(':')[-1].strip()))
        
len(geo_error)

In [ ]:
# find lines which start with e.g. '1:', like '2: david, holes_david_shape_11', get the name

file_names = []

for number in range(len(geo_error)):
    for line in lines:
        if str(number) + ':' in line:
            file_names.append(line.split(',')[1].strip())
            break
        
len(file_names)

In [ ]:
import trimesh
from tqdm import tqdm

face_areas = []

for mesh_file in tqdm(file_names):
    
    mesh = trimesh.load_mesh(
        f'/home/s94zalek_hpc/shape_matching/data/SHREC16_test/holes/off/{mesh_file}.off',
                             process=False, validate=True)
    
    # print(f'area_faces: {mesh.area_faces.sum()}')
    
    face_areas.append(mesh.area_faces.sum())
    
face_areas
    

In [ ]:
import numpy as np

print(f'no zoomout: {np.mean(geo_error[:101])}')
print(f'zoomout: {np.mean(geo_error[101:])}')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 4))

ax[0].scatter(face_areas, geo_error[:101])
ax[0].set_yscale('log')
ax[0].set_title('no zoomout')

ax[1].scatter(face_areas, geo_error[101:])
ax[1].set_yscale('log')
ax[1].set_title('zoomout')
plt.show()

In [ ]:
# get indices where geo_error[101:] is < 10
best_meshes = [i for i, error in enumerate(geo_error[101:]) if error < 20]
best_meshes

In [ ]:
[face_areas[i] for i in best_meshes]

In [42]:
scene = trimesh.Scene()

In [ ]:
scene.geometry.clear()

for curr_idx, i in enumerate(best_meshes):
    
    mesh_file = file_names[i]
    
    mesh = trimesh.load_mesh(
        f'/home/s94zalek_hpc/shape_matching/data/SHREC16_test/holes/off/{mesh_file}.off',
                             process=False, validate=True)
    
    mesh.vertices += np.array([curr_idx, 0, 0])
    
    scene.add_geometry(mesh)
    
scene.show()
    

In [ ]:
worst_meshes = [i for i, error in enumerate(geo_error[101:]) if error > 20]
len(worst_meshes)

In [ ]:
scene.geometry.clear()

for curr_idx, i in enumerate(worst_meshes):
    
    mesh_file = file_names[i]
    
    mesh = trimesh.load_mesh(
        f'/home/s94zalek_hpc/shape_matching/data/SHREC16_test/holes/off/{mesh_file}.off',
                             process=False, validate=True)
    
    mesh.vertices += np.array([curr_idx, 0, 0])
    
    scene.add_geometry(mesh)
    
scene.show()
    

In [88]:
base_dir = '/home/s94zalek_hpc/shape_matching/SLURM_logs'

# remove all files from base_dir that start with 'cache_surreal'
import os

for file in os.listdir(base_dir):
    if file.startswith('cache_surreal'):
        # print(file)
        os.remove(os.path.join(base_dir, file)) 